# Naive Bayes classifier for Fake News recognition
Fake news are defined by the New York Times as *"a made-up story with an intention to deceive"*, with
the intent to confuse or deceive people. They are everywhere in our daily life, and come especially from
social media platforms and applications in the online world. Being able to distinguish fake contents form
real news is today one of the most serious challenges facing the news industry.  

[Naive Bayes classifiers][1] are powerful algorithms that are used for text data analysis and are connected
to classification tasks of text in multiple classes.  
The goal of the project is to implement a __Multinomial Naive Bayes classifier__ in `R` and test its perfor-
mances in the classification of social media posts.
The suggested data set is available on [Kaggle][2]

[1]: https://nlp.stanford.edu/IR-book/pdf/13bayes.pdf "C. D. Manning, Chapter 13, Text Classification and Naive Bayes, in Introduction to Information Retrieval, Cambridge University Press, 2008."
[2]: https://www.kaggle.com/datasets/anmolkumar/fake-news-content-detection?select=train.csv

## Import useful libraries

In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Import dataset

In [3]:
df <- read_csv('train.csv', col_types = 'icc')
head(df, 10)

Labels,Text,Text_Tag
<int>,<chr>,<chr>
1,Says the Annies List political group supports third-trimester abortions on demand.,abortion
2,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments"
3,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy
1,Health care reform legislation is likely to mandate free sex change surgeries.,health-care
2,The economic turnaround started at the end of my term.,"economy,jobs"
5,The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.,education
0,Jim Dunnam has not lived in the district he represents for years now.,candidates-biography
2,"I'm the only person on this stage who has worked actively just last year passing, along with Russ Feingold, some of the toughest ethics reform since Watergate.",ethics
2,"However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations Center-Pacific.",jobs


## Multinomial NB classifier implementation

In [36]:
accuracy <- function( predictions, true_labels) {
    sum(predictions==true_labels) / length(predictions)
}

In [4]:
extract_vocabulary <- function(documents, unique=TRUE) {
    words <- unlist(str_split(documents, ' '))
    words <- str_replace_all(words, "[:punct:]|[$]", "")
    words <- words[words!='']
    words <- casefold(words, upper = FALSE)
    words <- str_replace(words, "[:digit:]+", "isnumeric")
    if(unique) return(unique(words))
    else return(words)
}

In [5]:
multinomNBC.fit <- function(documents, labels) {
    # documents must be a list

    
    log.like  <- list()

    # extract vocabulary
    V <- extract_vocabulary(documents[[1]])
    N <- length(documents[[1]])
    log.prior <- log(table(labels)) - log(N)

    for(l in unique(labels[[1]])) {
        
        text <- extract_vocabulary(documents[labels==l], unique=FALSE)
        freq <- table(c(text, V))
        log.like[[as.character(l)]] <- log(freq) - log(sum(freq))
    }

    return(list(log.likelihood=log.like, log.prior=log.prior, vocabulary=V))
}

In [59]:
multinomNBC.predict <- function(doc, log.prior, log.likelihood, voc){

    text <- extract_vocabulary(doc, unique=FALSE)
    freq <- table(text)[voc]
    freq <- freq[!is.na(freq)]
    #print(freq)
    scores <- rep(0, length(log.prior))
    names(scores) <- dimnames(log.prior)$labels

    for( l in names(scores)) {
        scores[l] <- log.prior[l] + sum(log.likelihood[[l]][dimnames(freq)$text] * freq)
    }
    
    return(names(scores)[which.max(scores)[[1]]])
}

In [6]:
nDoc <- 500

results <- multinomNBC.fit(df[1:nDoc, 2], df[1:nDoc, 1])

 'table' num [1:2366(1d)] -4.99 -8.24 -8.24 -8.24 -8.24 ...
 - attr(*, "dimnames")=List of 1
  ..$ : chr [1:2366] "a" "abbas" "abbott" "abdul" ...


In [58]:
multinomNBC.predict(df[1, 2][[1]], results$log.prior, results$log.likelihood, results$vocabulary)

text
          says            the         annies           list      political 
             1              1              1              1              1 
         group       supports thirdtrimester      abortions             on 
             1              1              1              1              1 
        demand 
             1 


[1] "1"

In [61]:
pred <- Vectorize(multinomNBC.predict, vectorize.args='doc', USE.NAMES=FALSE)(df[1:1000, 2][[1]], results$log.prior, results$log.likelihood, results$vocabulary)

accuracy(pred, df[1:1000, 1][[1]])

[1] 0.564

In [30]:
?Vectorize

Vectorize                 package:base                 R Documentation

_V_e_c_t_o_r_i_z_e _a _S_c_a_l_a_r _F_u_n_c_t_i_o_n

_D_e_s_c_r_i_p_t_i_o_n:

     ‘Vectorize’ creates a function wrapper that vectorizes the action
     of its argument ‘FUN’.

_U_s_a_g_e:

     Vectorize(FUN, vectorize.args = arg.names, SIMPLIFY = TRUE,
               USE.NAMES = TRUE)
     
_A_r_g_u_m_e_n_t_s:

     FUN: function to apply, found via ‘match.fun’.

vectorize.args: a character vector of arguments which should be
          vectorized.  Defaults to all arguments of ‘FUN’.

SIMPLIFY: logical or character string; attempt to reduce the result to
          a vector, matrix or higher dimensional array; see the
          ‘simplify’ argument of ‘sapply’.

USE.NAMES: logical; use names if the first ... argument has names, or
          if it is a character vector, use that character vector as the
          names.

_D_e_t_a_i_l_s:

     The arguments name

In [277]:
log(table(setdiff(voc, text)))


       abortion          across             act        actively           added 
              0               0               0               0               0 
     affordable          aflcio africanamerican           after         against 
              0               0               0               0               0 
            ago          agrees            alex             all          almost 
              0               0               0               0               0 
          along         already         america        american    americanmade 
              0               0               0               0               0 
      americans              an          anyone        anywhere         appears 
              0               0               0               0               0 
            arm          around              as     atlantaarea       authority 
              0               0               0               0               0 
     authorized            

In [258]:
#our vocabulary
voc <- extract_vocabulary(df[1:100, 2][[1]])
voc

[1] "says"            "the"             "annies"          "list"           
  [5] "political"       "group"           "supports"        "thirdtrimester" 
  [9] "abortions"       "on"              "demand"          "when"           
 [13] "did"             "decline"         "of"              "coal"           
 [17] "start"           "it"              "started"         "natural"        
 [21] "gas"             "took"            "off"             "that"           
 [25] "to"              "begin"           "in"              "president"      
 [29] "george"          "w"               "bushs"           "administration" 
 [33] "hillary"         "clinton"         "agrees"          "with"           
 [37] "john"            "mccain"          "by"              "voting"         
 [41] "give"            "bush"            "benefit"         "doubt"          
 [45] "iran"            "health"          "care"            "reform"         
 [49] "legislation"     "is"              "likely"          "mandate"        
 [53] "free"            "sex"             "change"          "surgeries"      
 [57] "economic"        "turnaround"      "at"              "end"            
 [61] "my"              "term"            "chicago"         "bears"          
 [65] "have"            "had"             "more"            "starting"       
 [69] "quarterbacks"    "last"            "isnumeric"       "years"          
 [73] "than"            "total"           "number"          "tenured"        
 [77] "uw"              "faculty"         "fired"           "during"         
 [81] "two"             "decades"         "jim"             "dunnam"         
 [85] "has"             "not"             "lived"           "district"       
 [89] "he"              "represents"      "for"             "now"            
 [93] "im"              "only"            "person"          "this"           
 [97] "stage"           "who"             "worked"          "actively"       
[101] "just"            "year"            "passing"         "along"          
[105] "russ"            "feingold"        "some"            "toughest"       
[109] "ethics"          "since"           "watergate"       "however"        
[113] "million"         "oregon"          "lottery"         "funds"          
[117] "port"            "newport"         "eventually"      "land"           
[121] "new"             "noaa"            "marine"          "operations"     
[125] "centerpacific"   "gop"             "primary"         "opponents"      
[129] "glenn"           "grothman"        "and"             "joe"            
[133] "leibham"         "cast"            "a"               "compromise"     
[137] "vote"            "cost"            "higher"          "electricity"    
[141] "costs"           "first"           "time"            "history"        
[145] "share"           "national"        "popular"         "margin"         
[149] "smaller"         "latino"          "nearly"          "americans"      
[153] "slipped"         "out"             "middle"          "class"          
[157] "into"            "poverty"         "mitt"            "romney"         
[161] "was"             "governor"        "massachusetts"   "we"             
[165] "didnt"           "slow"            "rate"            "growth"         
[169] "our"             "government"      "actually"        "cut"            
[173] "economy"         "bled"            "billion"         "due"            
[177] "shutdown"        "most"            "affordable"      "act"            
[181] "already"         "sense"           "been"            "waived"         
[185] "or"              "otherwise"       "suspended"       "election"       
[189] "november"        "percent"         "american"        "people"         
[193] "chose"           "young"           "lowincome"       "workers"        
[197] "opposed"         "requirement"     "buy"             "americanmade"   
[201] "motorcycles"     "said"            "all"             "buyamerican"    
[205] "provisions"      

In [115]:
example_char <- c("ciao sono Pietro ho 23 anni e non 50 ")
example <- str_split(example_char,' ')[[1]]
num <- 0
for (i in 1:length(example)){
    # print(example[i])
    aa <- as.numeric((example[i]))
    # print(is.na(aa))
    if(is.na(aa)==FALSE) {
        num <- num + 1
    }
}
cat("\nIn the sentences there are",num,"numbers")

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In the sentences there are 2 numbers

In [11]:
str_split(df[2:100, 2], ' ')

Warning message in stri_split_regex(string, pattern, n = n, simplify = simplify, :
“argument is not an atomic vector; coercing”


[[1]]
   [1] "c(\"When"           "did"                "the"               
   [4] "decline"            "of"                 "coal"              
   [7] "start?"             "It"                 "started"           
  [10] "when"               "natural"            "gas"               
  [13] "took"               "off"                "that"              
  [16] "started"            "to"                 "begin"             
  [19] "in"                 "(President"         "George"            
  [22] "W.)"                "Bushs"              "administration.\","
  [25] "\"Hillary"          "Clinton"            "agrees"            
  [28] "with"               "John"               "McCain"            
  [31] "\\\"by"             "voting"             "to"                
  [34] "give"               "George"             "Bush"              
  [37] "the"                "benefit"            "of"                
  [40] "the"                "doubt"              "on"                
  [43] "Iran.\\\"\","       "\"Health"           "care"              
  [46] "reform"             "legislation"        "is"                
  [49] "likely"             "to"                 "mandate"           
  [52] "free"               "sex"                "change"            
  [55] "surgeries.\","      "\"The"              "economic"          
  [58] "turnaround"         "started"            "at"                
  [61] "the"                "end"                "of"                
  [64] "my"                 "term.\","           "\"The"             
  [67] "Chicago"            "Bears"              "have"              
  [70] "had"                "more"               "starting"          
  [73] "quarterbacks"       "in"                 "the"               
  [76] "last"               "10"                 "years"             
  [79] "than"               "the"                "total"             
  [82] "number"             "of"                 "tenured"           
  [85] "(UW)"               "faculty"            "fired"             
  [88] "during"             "the"                "last"              
  [91] "two"                "decades.\","        "\n\"Jim"           
  [94] "Dunnam"             "has"                "not"               
  [97] "lived"              "in"                 "the"               
 [100] "district"           "he"                 "represents"        
 [103] "for"                "years"              "now.\","           
 [106] "\"I'm"              "the"                "only"              
 [109] "person"             "on"                 "this"              
 [112] "stage"              "who"                "has"               
 [115] "worked"             "actively"           "just"              
 [118] "last"               "year"               "passing,"          
 [121] "along"              "with"               "Russ"              
 [124] "Feingold,"          "some"               "of"                
 [127] "the"                "toughest"           "ethics"            
 [130] "reform"             "since"              "Watergate.\","     
 [133] "\"However,"         "it"                 "took"              
 [136] "$19.5"              "million"            "in"                
 [139] "Oregon"             "Lottery"            "funds"             
 [142] "for"                "the"                "Port"              
 [145] "of"                 "Newport"            "to"                
 [148] "eventually"         "land"               "the"               
 [151] "new"                "NOAA"               "Marine"            
 [154] "Operations"         "Center-Pacific.\"," "\"Says"            
 [157] "GOP"                "primary"            "opponents"         
 [160] "Glenn"              "Grothman"           "and"               
 [163] "Joe"                "Leibham"            "cast"              
 [166] "a"                  "compromise"         "vote"              
 [169] "that"               "cost"               "$788"              
 [17

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=811d2f44-4ec2-4c23-800a-72ed1acf812e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>